In [17]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [ ]:
base_path = '/home/cburich_pymnts/buckets/b1/'
dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'


# base_path = 'C:/Users/Cristian Burich/Desktop/MA/segundo/eyf/'
# dataset_path = base_path + 'datasets/'
# modelos_path = base_path + 'modelos/'
# db_path = base_path + 'db/'


dataset_file = 'competencia_03_fe_U_k300.parquet' 

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [165229,165211,165203,165237,165247]

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [ ]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202002, 202003, 202004, 202005,
               202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105, 202106] # dejo afuera 202107 para test

data.shape

(376300, 679)

In [21]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [22]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [ ]:
train_data = data[data['foto_mes'].isin(meses_train)]

X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2', 'foto_mes'], axis=1)
y_train_binaria1 = train_data['clase_binaria1'] # Solo BAJA+2
y_train_binaria2 = train_data['clase_binaria2'] # junta a los 2 BAJA
w_train = train_data['clase_peso']

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases *BAJA+1* y *BAJA+2* en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el *weight*. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.


In [24]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Entrenamiento

Cargamos el study de optuna que optimizamos en el script anterior

In [ ]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "competencia3_lgbm_k300" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-18 07:42:27,674] Using an existing study with name 'competencia2_lgbm_v08' instead of creating a new one.


In [26]:
resultados = study.trials_dataframe()
resultados.shape

(100, 12)

Entrenamiento:

In [ ]:
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")

params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'seed': semillas[0],
    'verbose': 0
}

train_data = lgb.Dataset(X_train,
                          label=y_train_binaria2,
                          weight=w_train)

model = lgb.train(params,
                  train_data,
                  num_boost_round=best_iter)


Mejor cantidad de árboles para el mejor model 871


In [29]:
params

{'objective': 'binary',
 'boosting_type': 'gbdt',
 'first_metric_only': True,
 'boost_from_average': True,
 'feature_pre_filter': False,
 'max_bin': 31,
 'num_leaves': 95,
 'learning_rate': 0.014790793124814124,
 'min_data_in_leaf': 1162,
 'feature_fraction': 0.32039319093779284,
 'bagging_fraction': 0.7236519946486292,
 'seed': 122219,
 'verbose': 0}

Variables mas importantes:

In [30]:
importances = model.feature_importance()
feature_names = X_train.columns.tolist()
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values('importance', ascending=False)
importance_df[importance_df['importance'] > 0]

,feature,importance
1,foto_mes,1767
5,cliente_edad,711
147,Visa_fechaalta,610
10,mactivos_margen,606
125,Master_fechaalta,595
...,...,...
431,slope_mcheques_depositados_rechazados,1
34,cprestamos_hipotecarios,1
618,slope_avg3_Master_madelantodolares,1
64,mcajeros_propios_descuentos,1


Para guardar el modelo para poder utilizarlo más adelante, no es necesario guardarlo como *pickle*, la librería nos permite guardarlo en formato texto

In [31]:
modelos_path

'C:/Eugenio/Maestria/DMEyF/modelos/'

### Entrenamos con 5 semillas y guardamos los modelos

In [ ]:
version = 'v008' # UPDATE

for semilla in semillas:
    
    best_iter = study.best_trial.user_attrs["best_iter"]
    print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
    
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': study.best_trial.params['num_leaves'],
        'learning_rate': study.best_trial.params['learning_rate'],
        'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
        'feature_fraction': study.best_trial.params['feature_fraction'],
        'bagging_fraction': study.best_trial.params['bagging_fraction'],
        'seed': semilla,
        'verbose': 0
    }

    train_data = lgb.Dataset(X_train,
                            label=y_train_binaria2,
                            weight=w_train)

    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    model.save_model(modelos_path + f'{version}/lgb_competencia2_{version}_s{semilla}.txt')

Mejor cantidad de árboles para el mejor model 871
Mejor cantidad de árboles para el mejor model 871
Mejor cantidad de árboles para el mejor model 871
Mejor cantidad de árboles para el mejor model 871
Mejor cantidad de árboles para el mejor model 871
